# Análisis exploratorio de datos🔍:

###DATASET **Glacier.csv**:
Es un conjunto de datos **sobre cambios en los glaciares** (longitud, área, volumen, masa) estandarizados y recopilados internacionalmente, basados ​​en observaciones in situ y teledetección, así como en reconstrucciones. publicado por World Glacier Monitoring Service [**WGMS**](https://wgms.ch/data_databaseversions)

- `✍creado` 2024-01-23
- `⏰temporalidad de los datos` 1127/2024

`
`
### 📄 Consideraciones a la hora de analizar `glacier.csv`:

+ este csv contiene Información general (y presumiblemente estática) sobre cada glaciar. Al enviar un nuevo glaciar, asigne un `WGMS_ID` temporal y utilícelo como `WGMS_ID` en todas las demás filas de la tabla que correspondan a este glaciar.

+ ⚠ `Atención` desde WGMS advierten que los datos pueden contener errores e inexactitudes y sugieren que en caso de ambigüedades se les comunique o bien a ellos o a los investigadores e instituciones enumerados en los datos (columnas `INVESTIGADOR` y `SPONS_AGENCY`).



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix


### Primero, leemos y exploramos que contiene el archivo `glacier.csv`
El archivo csv se encuentra guardado dentro de nuestro Drive en una carpeta llamada `data_saturdays` /content/drive/MyDrive/data_saturdays/glacier.csv

In [ ]:
#leemos el archivo y representamos sus primeras 10 líneas
df_glaciers = pd.read_csv('/content/drive/MyDrive/data_saturdays/glacier.csv')
df_glaciers.head(10)

<ipython-input-24-c6e9d99e5448>:2: DtypeWarning: Columns (3,4,10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_glaciers = pd.read_csv('/content/drive/MyDrive/data_saturdays/glacier.csv')


,POLITICAL_UNIT,NAME,WGMS_ID,GEN_LOCATION,SPEC_LOCATION,LATITUDE,LONGITUDE,PRIM_CLASSIFIC,FORM,FRONTAL_CHARS,EXPOS_ACC_AREA,EXPOS_ABL_AREA,PARENT_GLACIER,REMARKS,GLACIER_REGION_CODE,GLACIER_SUBREGION_CODE
0,AF,PIR YAKH,10452,UPPER CABUL RIVER BASIN,CHUMAR VALLEY,35.595001,70.169998,6.0,3.0,8.0,NE,NE,NaN,Local people call this glacier PIR YAKH which ...,ASW,ASW-01
1,AF,RGI60-13.10763,156232,NaN,NaN,37.262384,73.525069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASC,ASC-02
2,AF,RGI60-13.10764,156233,NaN,NaN,37.258539,73.546610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASC,ASC-02
3,AF,RGI60-13.10765,156234,NaN,NaN,37.266134,73.542243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASC,ASC-02
4,AF,RGI60-13.10766,156235,NaN,NaN,37.273266,73.535659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASC,ASC-02
5,AF,RGI60-13.10767,156236,NaN,NaN,37.267820,73.557927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASC,ASC-02
6,AF,RGI60-13.10768,156237,NaN,NaN,37.270174,73.566393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASC,ASC-02
7,AF,RGI60-13.10769,156238,NaN,NaN,37.273807,73.576475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASC,ASC-02
8,AF,RGI60-13.10770,156239,NaN,NaN,37.284926,73.579521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASC,ASC-02
9,AF,RGI60-13.10771,156240,NaN,NaN,37.290455,73.565636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASC,ASC-02


### Sacamos el nombre de las **columnas** para comprender que datos contiene este dataset

In [ ]:
df_glaciers.columns.values

array(['POLITICAL_UNIT', 'NAME', 'WGMS_ID', 'GEN_LOCATION',
       'SPEC_LOCATION', 'LATITUDE', 'LONGITUDE', 'PRIM_CLASSIFIC', 'FORM',
       'FRONTAL_CHARS', 'EXPOS_ACC_AREA', 'EXPOS_ABL_AREA',
       'PARENT_GLACIER', 'REMARKS', 'GLACIER_REGION_CODE',
       'GLACIER_SUBREGION_CODE'], dtype=object)

Ahora que ya las conocemos vamos a explicar cada una de ellas:
#### `👉POLITICAL_UNIT`:
+ Código de dos caracteres indica el **país en el que se encuentra el glaciar**. Una lista de códigos está disponible en https://www.iso.org/obp/ui/#search/code.

#### `👉NAME`:
+ Nombre del glaciar

#### `👉WGMS_ID`:
+ identificador de los glaciares en la base de datos de Fluctuaciones de Glaciares (FoG)

#### `👉GEN_LOCATION`:
+ Se refiere a una **entidad geográfica grande** da una idea aproximada de la ubicación del glaciar, sin requerir el uso de un mapa o un atlas. (ejemplo: gran cadena montañosa)

#### `👉SPEC_LOCATION`:
+ Se refiere a una **ubicación geográfica más específica** que se puede encontrar fácilmente en un mapa a pequeña escala. ejemplo: (ejemplo: subcordillera montañosa )


#### `👉LATITUDE`:
+ Latitud en grados decimales (°, WGS 84).
++ valores positivos = hemisferio norte
++ valores negativos= hemisferio sur

#### `👉LONGITUDE`:
+ Longitud en grados decimales (°, WGS 84).
++ valores positivos = el este del meridiano cero
++ valores negativos= el oeste del meridiano cero


#### `👉PRIM_CLASSIFIC`:
 + Clasificación de los glaciares según Masas perennes de hielo y nieve según [UNESCO/IAHS, 1970](https://www.wgms.ch/downloads/UNESCO_1970.pdf)

      - `0` = Otro
      - `1` = Capa de hielo continental
      - `2`= Campo de Hielo
      - `3`= Capa de hielo
      - `4` = Glaciar de salida.
      - `5`= Glaciar del valle
      - `6` = Glaciar de montaña
      - `7`= Glaciares y campos de nieve
      - `8`= Plataforma de hielo
      - `9`= Glaciar de roca

#### `👉FORM`:
  + **Clasificación** de los glaciares según **su forma** según Masas perennes de hielo y nieve [UNESCO/IAHS, 1970](https://www.wgms.ch/downloads/UNESCO_1970.pdf)

      - `0` = Otro
      - `1` = Cuencas compuestas
      - `2` = Cuenca compuesta
      - `3` = Cuenca simple
      - `4` = Circo
      - `5`= Nicho
      - `6` = Cráter
      - `7`= Plata de hielo
      - `8`= Grupo
      - `9`= Remanente


#### `👉FRONTAL_CHARS`:
  + **Clasificación** de los glaciares según **su frontal** según Masas perennes de hielo y nieve [UNESCO/IAHS, 1970](https://www.wgms.ch/downloads/UNESCO_1970.pdf)

      - `0` = Otro
      - `1` = Piamonte
      - `2` = pie expandido
      - `3` = Lobulado
      - `4` = Desprendimiento
      - `5`= Coalescente
      - `6` = Hielo irregular, principalmente limpio
      - `7`= Irregular, cubierto de escombros
      - `8`= Un solo lóbulo, principalmente hielo limpio
      - `9`= Un solo lóbulo, cubierto de escombros


#### `👉EXPOS_ACC_AREA`:

  + **Orientación** principal del **área de acumulación** mediante una brújula de 8 puntos.

#### `👉EEXPOS_ABL_AREA`

  + **Orientación** principal de la **zona de ablación** mediante brújula de 8 puntos.

#### `👉PARENT_GLACIER`

  + es un `WGMS_ID` que se utiliza para **vincular los glaciares** con su  glaciar padre.

#### `👉REMARKS`

  + algo así como **comentarios** importantes no incluido en otro lugar.



#### `👉GLACIER_REGION_CODE`

  + **Código de la región** en la que se encuentra el glaciar según [*Glacier Regions* (GTN-G, 2017)](https://doi.org/10.5904/gtng-glacreg-2017-07)


#### `👉GLACIER_REGION_CODE`

+ **Código de la subregión** en la que se encuentra el glaciar según [*Glacier Regions* (GTN-G, 2017)](https://doi.org/10.5904/gtng-glacreg-2017-07)
  



  

# Conclusión
Este dataset `glacier.csv` de momento no arroja variables que nos ayuden a predecir la masa de los glaciares, las columnas que hacen referencia a estos aspectos no dan un dato medible, o bien nos da una orientación o una clasificación del uno al 9 según el tipo de glaciar. Vamos a estudiar el resto de dataset del conjunto.


